In [105]:
import sys
import time
import re

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [112]:
def lookup_cik(company_name):
    company_name = company_name.replace(' ', '%20')
    cik_lookup = f'https://www.sec.gov/cgi-bin/browse-edgar?company={company_name}'
    r = requests.get(cik_lookup)
    try:
        return re.search(r'CIK=(\d*)&amp', r.text)[1]
    except:
        return "Error!"

In [115]:
cik = lookup_cik('alphabet inc')
print(cik)

0001652044


In [63]:
# Access page
# cik = '0001652044'
doctype = '10-K'
dateb = '20200101' #Search 

In [64]:
# Obtain HTML for search page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
edgar_resp = requests.get(base_url.format(cik, doctype, dateb))
edgar_str = edgar_resp.text

In [65]:
edgar_resp.url

'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001652044&type=10-K&dateb=20200101'

In [89]:
# Find the document link
doc_link = ''
yr = dateb[:2] + str(int(dateb[2:4])-1)
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        print(cells[3].text, cells[1].a['href'])
        if yr in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']

2019-02-06 /Archives/edgar/data/1652044/000119312519028757/0001193125-19-028757-index.htm
2019-02-05 /Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004-index.htm
2018-02-06 /Archives/edgar/data/1652044/000165204418000007/0001652044-18-000007-index.htm
2017-02-03 /Archives/edgar/data/1652044/000165204417000008/0001652044-17-000008-index.htm
2016-03-29 /Archives/edgar/data/1652044/000119312516520367/0001193125-16-520367-index.htm
2016-02-11 /Archives/edgar/data/1652044/000165204416000012/0001652044-16-000012-index.htm


In [90]:
doc_link

'https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004-index.htm'

In [91]:
# Obtain HTML for document page
doc_resp = requests.get(doc_link)
doc_str = doc_resp.text

In [92]:
# Find the XBRL link
xbrl_link = ''
soup = BeautifulSoup(doc_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile', summary='Data Files')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if 'INS' in cells[3].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']

In [93]:
# Obtain XBRL text from document
xbrl_resp = requests.get(xbrl_link)
xbrl_str = xbrl_resp.text

In [121]:
# Find and print stockholder's equity
amts = {}
soup = BeautifulSoup(xbrl_str, 'lxml')
tag_list = soup.find_all()

line = None

for num, tag in enumerate(tag_list):
    if tag.name.startswith('us-gaap'):
        
        if '<div>' in tag.text:
            continue
        
        if line:
            prev_line = str(line)
        
        line = tag.name.split(':')[-1]
        
        
        period = tag.get('contextref').split('_Entity')[0].split('_dei')[0].split('_us-gaap')[0]
        
        if '<div' in tag.text:
            continue
        
#         try:
#             amt = int(tag.text) * 10**(int(tag.get('decimals')))
#         except:
#             amt = tag.text
        
        amt = tag.text
    
#         print(tag)
        print(f"{line} {period}: {amt}")
        
        time.sleep(.001)

accountspayablecurrent FI2017Q4: 3137000000
accountspayablecurrent FI2018Q4: 4378000000
accountsreceivablenetcurrent FI2017Q4: 18336000000
accountsreceivablenetcurrent FI2018Q4: 20838000000
accruedincometaxescurrent FI2017Q4: 881000000
accruedincometaxescurrent FI2018Q4: 69000000
accruedincometaxesnoncurrent FI2017Q4: 12812000000
accruedincometaxesnoncurrent FI2018Q4: 11327000000
accruedliabilitiescurrent FI2017Q4: 10177000000
accruedliabilitiescurrent FI2018Q4: 16958000000
accumulateddepreciationdepletionandamortizationpropertyplantandequipment FI2017Q4: 17264000000
accumulateddepreciationdepletionandamortizationpropertyplantandequipment FI2018Q4: 22788000000
accumulatedothercomprehensiveincomelossnetoftax FI2017Q4: -992000000
accumulatedothercomprehensiveincomelossnetoftax FI2018Q4: -2306000000
acquiredfinitelivedintangibleassetsweightedaverageusefullife FD2017Q4YTD: P3Y8M
acquiredfinitelivedintangibleassetsweightedaverageusefullife FD2017Q4YTD: P8Y9M
acquiredfinitelivedintangibleass

In [20]:
tag

<us-gaap:writeoffofdeferreddebtissuancecost contextref="Duration_11_20_2014_To_11_20_2014_dei_LegalEntityAxis_alpx_NBTYIncMember_us-gaap_LongtermDebtTypeAxis_us-gaap_RevolvingCreditFacilityMember" decimals="-3" unitref="Unit1">359000</us-gaap:writeoffofdeferreddebtissuancecost>

In [22]:
10 ** -3

0.001

In [25]:
tag.decimals

In [27]:
int(tag.text) * 10**(-int(tag.get('decimals')))

359000000